<a href="https://colab.research.google.com/github/ihebakermi10/langgraph/blob/main/langgraph_scraping_nehos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
! pip install -U langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph

In [49]:
import getpass
import os




In [26]:


from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0,api_key=OPENAI_API_KEY)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

system = (
    "Vous êtes un évaluateur de contenu chargé de détecter si un document contient AU MOINS UNE de ces informations essentielles : "
    "horaires d'ouverture (explicites ou implicites), prix/tarifs, menu/catalogue, localisation (adresse, repères géographiques) "
    "ou email de contact. \n\n"

    "**Consignes clés** :\n"
    "1. Répondez 'Oui' dès qu'UNE SEULE information est identifiée (même indirectement).\n"
    "2. Acceptez les formulations contextuelles (ex: 'ouvert jusqu'à minuit', 'menu à 15€', 'proche de la gare').\n"
    "3. Répondez 'Non' UNIQUEMENT si le document est totalement générique (ex: 'Nous offrons un service professionnel').\n\n"

    "**Format de réponse** :\n"
    "- Si Oui : 'Oui [éléments détectés entre crochets]' (ex: Oui [horaires, prix])\n"
    "- Si Non : 'Non [raison en 3 mots]' (ex: Non [données absentes])\n\n"

    "Priorisez la pertinence pratique plutôt que l'exhaustivité."
)
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n "),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
docs = """
Restaurant La Belle Table
Horaires d'ouverture :
- Lundi à Vendredi : 11h30 - 14h30 / 19h - 23h
- Samedi : 12h - 00h
- Dimanche : Fermé

"""
print(retrieval_grader.invoke({ "document": docs}))

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1362: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


binary_score='yes'


In [35]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


# Nouveau modèle de sortie structurée pour le nettoyage
class CleanedDocument(BaseModel):
    """Document nettoyé avec les éléments clés extraits."""
    menu: str | None = Field(default=None, description="Menu/catalogue détecté")
    prix: str | None = Field(default=None, description="Prix/tarifs mentionnés")
    localisation: str | None = Field(default=None, description="Adresse complète ou coordonnées GPS")
    email: str | None = Field(default=None, description="Email de contact")
    telephone: str | None = Field(default=None, description="Numéro de téléphone")
    metadata: dict = Field(default_factory=dict, description="Métadonnées complémentaires")

# Instanciation du LLM en utilisant la clé API
llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0, api_key=OPENAI_API_KEY)
structured_llm_cleaner = llm.with_structured_output(CleanedDocument)

# Prompt d'extraction structurée
system_cleaning = (
    "Vous êtes un assistant de nettoyage de documents. Extrayez et structurez les éléments suivants :\n"
    "1. Menu/Catalogue : Liste des plats/produits avec descriptions\n"
    "2. Prix : Tarifs spécifiques ou fourchettes de prix\n"
    "3. Localisation : Adresse exacte, points de repère ou coordonnées GPS\n"
    "4. Email : Adresse email professionnelle\n"
    "5. Téléphone : Numéro de contact formaté\n\n"
    "**Règles d'extraction** :\n"
    "- Conservez les formats originaux pour les prix (ex: 'Entrées : 8-12€')\n"
    "- Pour les adresses, gardez le format complet (ex: '12 Rue du Marché, 75001 Paris')\n"
    "- Si un élément est absent, laissez le champ vide\n"
    "- Ajoutez dans metadata['elements_detectes'] la liste des éléments trouvés\n\n"
    "Exemple de sortie :\n"
    "menu: 'Entrées\\n- Salade César (12€)\\nPlats\\n- Boeuf Bourguignon (24€)'\n"
    "prix: 'Comptez 25-40€ par personne'\n"
    "localisation: '12 Rue des Oliviers, Marseille'\n"
    "metadata: {{'elements_detectes': ['menu', 'prix', 'localisation']}}"
)

clean_prompt = ChatPromptTemplate.from_messages([
    ("system", system_cleaning),
    ("human", "Document à nettoyer :\n\n{document}\n\nStructurez les éléments demandés :")
])

document_cleaner = clean_prompt | structured_llm_cleaner

doc_to_clean = ""
result = document_cleaner.invoke({"document": doc_to_clean})

# Affichage du résultat
print("=== Document nettoyé ===")
print(result)


/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1362: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


=== Document nettoyé ===
menu='## Choisissez votre burger !\n\nToulouse Burger vous propose de commander votre burger à Emporter, à manger sur Place et aussi en Livraison. Réservez dès à présent votre burger maison !\n\n## Les Menus\n\nLe Menu est composé du burger, des frites OU beignets d’oignons, de la sauce choisie et de la boisson.\n\n_Supplément Steak : 3,00€_\n\n_Supplément Chicken : 3,00€_\n\n_Supplément Galette : 2,50€_\n\nBURGER SIMPLE : 10€ (et 10,50€ en livraison)\n\nMENU SIMPLE : 15,50€\n\n## Nos burgers\n\n**Le Burger du Moment**\n\nDemandez la recette du mois\n\n**Le Classique** _(avec ou sans bacon)_\n\nSteak, Cheddar, Salade, Tomate, Oignons Rouges, Cornichons, Mayonnaise et Ketchup\n\n**Le Fred**_(création d’un client)_\n\nSteak, Tomme des Pyrénées, Mayonnaise au Piment d’Espelette Maison, Galette de Pomme de Terre, Confiture de Cerise Noire\n\n**Le Végétarien**\n\nTous nos burgers peuvent se transformer en Végétarien. Nous remplaçons le steak par une des trois galett

In [41]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
import os


class GradeInformationLoss(BaseModel):
    """
    Ce modèle renvoie un score binaire indiquant si le document nettoyé
    conserve toutes les informations pertinentes de l'original.

    binary_score:
        - 'yes' : Le document nettoyé conserve toutes les informations essentielles.
        - 'no' : Des informations importantes ont été perdues lors du nettoyage.
    """
    binary_score: str = Field(
        description="Indique si le document nettoyé conserve les infos pertinentes ('yes') ou non ('no')"
    )

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=OPENAI_API_KEY)
structured_llm_checker = llm.with_structured_output(GradeInformationLoss)

system_prompt = (
    "Vous êtes un évaluateur qui doit comparer un document original et sa version nettoyée. "
    "Votre tâche est de déterminer si le document nettoyé conserve toutes les informations essentielles "
    "(menu, prix, localisation, email, téléphone, etc.) présentes dans l'original. \n\n"
    "Si le document nettoyé contient bien toutes les informations nécessaires, répondez par 'yes'. "
    "Sinon, si certaines informations importantes manquent, répondez par 'no'."
)

grading_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "Document original :\n\n{original_document}\n\nDocument nettoyé :\n\n{cleaned_document}\n\n"
                "Le document nettoyé conserve-t-il toutes les informations pertinentes ? Répondez 'yes' ou 'no'.")
])

# Chaîne de vérification en combinant le prompt et le LLM structuré
information_loss_checker = grading_prompt | structured_llm_checker

# Exemple d'utilisation
original_document = """
Restaurant La Belle Table
Horaires d'ouverture :
- Lundi à Vendredi : 11h30 - 14h30 / 19h - 23h
- Samedi : 12h - 00h
- Dimanche : Fermé

Menu du Jour :
Entrées :
- Velouté de champignons (9€)
- Salade de chèvre chaud (12€)

Plats Principaux :
- Boeuf Bourguignon (24€)
- Daurade royale grillée (28€)

Contact : reservation@labelletable.com - Tel: 04 91 12 34 56

Adresse : 12 Cours Honoré d'Estienne d'Orves, 13001 Marseille
"""

cleaned_document = """
Restaurant La Belle Table

Menu du Jour :
- Velouté de champignons (9€)
- Salade de chèvre chaud (12€)
- Boeuf Bourguignon (24€)
- Daurade royale grillée (28€)

Contact : reservation@labelletable.com

Adresse : 12 Cours Honoré d'Estienne d'Orves, 13001 Marseille
"""

result = information_loss_checker.invoke({
    "original_document": original_document,
    "cleaned_document": cleaned_document
})

print("Score de conservation des informations :", result.binary_score)


/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1362: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1375: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-0125 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


Score de conservation des informations : no


In [45]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
import os


# Modèle de sortie pour la réécriture d'un document
class DocumentRewriterOutput(BaseModel):
    """
    Ce modèle renvoie le document réécrit avec toutes les informations essentielles complétées.

    rewritten_document:
        Le document réécrit contenant le menu complet, les tarifs, les coordonnées et l'adresse.
    """
    rewritten_document: str = Field(
        description="Document réécrit après complétion des informations manquantes"
    )

# Instanciation du LLM avec la clé API
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=OPENAI_API_KEY)
structured_llm_rewriter = llm.with_structured_output(DocumentRewriterOutput)

# Définition du prompt de réécriture
system_prompt = (
    "Vous êtes un assistant spécialisé dans la réécriture de documents de restaurant. "
    "Votre tâche est de prendre un document nettoyé qui pourrait manquer d'informations essentielles "
    "telles que le menu complet, les prix, les coordonnées (email, téléphone) et l'adresse, "
    "et de le réécrire en complétant ces éléments manquants de manière cohérente et naturelle. \n\n"
    "Assurez-vous de conserver le style et la structure du document tout en ajoutant les informations nécessaires."
)

rewrite_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "Document nettoyé incomplet :\n\n{document}\n\nRéécrivez le document en ajoutant toutes les informations essentielles (menu complet, tarifs, coordonnées, adresse) si elles manquent.")
])

# Chaîne de réécriture combinant le prompt, le LLM et le parser de sortie
document_rewriter = rewrite_prompt | structured_llm_rewriter

# Exemple d'utilisation
doc_clean = """
Restaurant La Belle Table

Menu du Jour :
- Velouté de champignons (9€)
- Salade de chèvre chaud (12€)
- Boeuf Bourguignon (24€)
- Daurade royale grillée (28€)

Contact : reservation@labelletable.com

Adresse : 12 Cours Honoré d'Estienne d'Orves, 13001 Marseille
"""

result = document_rewriter.invoke({"document": doc_clean})

print("=== Document réécrit ===")
print(result.rewritten_document)


/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1362: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1375: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-0125 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


=== Document réécrit ===
Restaurant La Belle Table

Menu du Jour :
- Velouté de champignons (9€)
- Salade de chèvre chaud (12€)
- Boeuf Bourguignon (24€)
- Daurade royale grillée (28€)

Contact : reservation@labelletable.com
Téléphone : +33 6 12 34 56 78

Adresse : 12 Cours Honoré d'Estienne d'Orves, 13001 Marseille


In [50]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# Instanciation du LLM (GPT-3.5-turbo ici)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=OPENAI_API_KEY)

###############################################################################
# 1️⃣ Classe de stockage de texte
###############################################################################

class StoreTextOutput(BaseModel):
    """
    Modèle de sortie pour le stockage de texte.

    stored_text:
        Le texte tel qu'il a été stocké.
    """
    stored_text: str = Field(..., description="Texte stocké avec succès")

store_text_system = (
    "Vous êtes un assistant de stockage de texte. Vous recevez un texte et vous devez le renvoyer tel quel "
    "dans le format JSON suivant : {{'stored_text': 'votre texte ici'}}."
)
store_text_prompt = ChatPromptTemplate.from_messages([
    ("system", store_text_system),
    ("human", "Veuillez stocker le texte suivant :\n\n{text}")
])

store_text_chain = store_text_prompt | llm.with_structured_output(StoreTextOutput)

input_text = "Ceci est le texte à stocker dans la variable."
result_store = store_text_chain.invoke({"text": input_text})
print("=== Résultat du stockage ===")
print(result_store)
print("\n")



/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1362: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1375: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-0125 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


=== Résultat du stockage ===
stored_text='Ceci est le texte à stocker dans la variable.'




/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1362: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1375: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-0125 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


=== Résultat de la vérification ===
exists='yes'


In [51]:

class VerifyDataOutput(BaseModel):
    """
    Modèle de sortie pour la vérification de présence de données.

    exists:
        'yes' si la donnée figure dans la base, 'no' sinon.
    """
    exists: str = Field(..., description="Indique si la donnée existe dans la base ('yes' ou 'no')")

verify_data_system = (
    "Vous êtes un assistant chargé de vérifier si une donnée donnée existe dans une base de données. "
    "La base de données est fournie sous forme d'une liste d'éléments (texte). "
    "Si la donnée figure dans la base, répondez 'yes'. Sinon, répondez 'no'."
)
verify_data_prompt = ChatPromptTemplate.from_messages([
    ("system", verify_data_system),
    ("human", "Donnée à vérifier : {data}\nBase de données : {database}\n"
                "La donnée existe-t-elle dans la base ? Répondez 'yes' ou 'no'.")
])

verify_data_chain = verify_data_prompt | llm.with_structured_output(VerifyDataOutput)

# Exemple d'utilisation de la chaîne de vérification
data_to_check = "Ceci est le texte à stocker dans la variable."  # On vérifie le même texte
# Simulation d'une base de données sous forme de liste
database = [
    "Autre texte",
    "Ceci est le texte à stocker dans la variable.",
    "Texte supplémentaire"
]

result_verify = verify_data_chain.invoke({"data": data_to_check, "database": database})
print("=== Résultat de la vérification ===")
print(result_verify)


/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1362: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1375: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-0125 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


=== Résultat de la vérification ===
exists='yes'
